# Datafaucet

Datafaucet is a productivity framework for ETL, ML application. Simplifying some of the common activities which are typical in Data pipeline such as project scaffolding, data ingesting, start schema generation, forecasting etc.

In [1]:
import datafaucet as dfc
dfc.__version__

'0.9.1'

## Loading and Saving Data

In [2]:
dfc.project.load()

NOTICE dfc engine.py:__init__ Connecting to spark master: local[*]
NOTICE dfc engine.py:__init__ Engine context spark:2.4.4 successfully started


In [3]:
def equal(a,b):
    cnt = a.exceptAll(b).count() + b.exceptAll(a).count()
    return cnt==0

def mask_rootdir(resource):
    d = resource.copy()
    if d['service']=='file':
        d['url'] = '<project_rootdir>/' + dfc.utils.relpath(d['url'], dfc.rootdir())
    return d

### Slow Changing Dimensions 


In [4]:
df = dfc.range(3)
df = df.cols.create('name').fake('first_name')
df = df.cols.create('amount').randint(10)
df = df.cols.create('purchase_date').fake('date_this_year')
df.cache()

df.data.grid(10)

,id,name,amount,purchase_date
0,0,Jaime,4,2019-07-19
1,1,Julian,3,2019-01-21
2,2,Angela,2,2019-08-25


In [5]:
# basic save with overwrite
df.save('data/saved/dim', format='scd:parquet', mode='overwrite')
dfc.list('data/saved/dim').data.grid()

,name,type
0,.part-00000-350b2931-b8e7-4a72-a2f2-4e5ac6eec5...,FILE
1,part-00000-350b2931-b8e7-4a72-a2f2-4e5ac6eec5f...,FILE
2,_SUCCESS,FILE
3,._SUCCESS.crc,FILE


In [6]:
# save with partition by 'purchase_date'
df.save('data/saved/dim', format='scd:parquet', mode='overwrite', partitionBy='purchase_date')
dfc.list('data/saved/dim').data.grid()

,name,type
0,purchase_date=2019-07-19,DIRECTORY
1,purchase_date=2019-08-25,DIRECTORY
2,_SUCCESS,FILE
3,purchase_date=2019-01-21,DIRECTORY
4,._SUCCESS.crc,FILE


In [7]:
dfc.load('data/saved/dim', format='scd:parquet').data.grid()

,id,name,amount,purchase_date
0,1,Julian,3,2019-01-21
1,2,Angela,2,2019-08-25
2,0,Jaime,4,2019-07-19


In [8]:
dfc.load('data/saved/dim', format='parquet').data.grid()

,id,name,amount,_state,_updated,purchase_date
0,1,Julian,3,0,2019-12-03 05:33:01,2019-01-21
1,2,Angela,2,0,2019-12-03 05:33:01,2019-08-25
2,0,Jaime,4,0,2019-12-03 05:33:01,2019-07-19


In [9]:
df = dfc.range(3)
df = df.cols.create('name').fake('first_name')
df = df.cols.create('amount').randint(10)
df = df.cols.create('purchase_date').fake('date_this_year')
df.cache()

df.data.grid(10)

,id,name,amount,purchase_date
0,0,Lori,2,2019-04-27
1,1,Stephanie,6,2019-02-06
2,2,Linda,6,2019-01-07


In [10]:
df.save('data/saved/dim', format='scd:parquet', mode='overwrite')
df_v0 = dfc.load('data/saved/dim', format='scd:parquet')
df_v0.data.grid()

,id,name,amount,purchase_date
0,0,Lori,2,2019-04-27
1,1,Stephanie,6,2019-02-06
2,2,Linda,6,2019-01-07


In [11]:
dfc.load('data/saved/dim', format='parquet').data.grid()

,id,name,amount,purchase_date,_state,_updated
0,0,Lori,2,2019-04-27,0,2019-12-03 05:33:04
1,1,Stephanie,6,2019-02-06,0,2019-12-03 05:33:04
2,2,Linda,6,2019-01-07,0,2019-12-03 05:33:04


In [12]:
# append one
from datetime import datetime
df = df.rows.append([{'id':3, 'name':'Steve', 'amount':8, 'purchase_date': datetime(2019,4,28)}])
df.data.grid()

,id,name,amount,purchase_date
0,0,Lori,2,2019-04-27
1,1,Stephanie,6,2019-02-06
2,2,Linda,6,2019-01-07
3,3,Steve,8,2019-04-28


In [13]:
df.save('data/saved/dim', format='scd:parquet', mode='append', merge_on='id')
df_v1 = dfc.load('data/saved/dim', format='scd:parquet', merge_on='id')
df_v1.data.grid()

NOTICE dfc engine.py:save_scd merge on=id, updated=0, added=1, deleted=0


,id,name,amount,purchase_date
0,0,Lori,2,2019-04-27
1,1,Stephanie,6,2019-02-06
2,2,Linda,6,2019-01-07
3,3,Steve,8,2019-04-28


In [14]:
# delete one
df = df.rows.delete('id = 2')
df.data.grid()

,id,name,amount,purchase_date
0,0,Lori,2,2019-04-27
1,1,Stephanie,6,2019-02-06
2,3,Steve,8,2019-04-28


In [15]:
df.save('data/saved/dim', format='scd:parquet', mode='append', merge_on='id')
df_v2 = dfc.load('data/saved/dim', format='scd:parquet', merge_on='id')
df_v2.data.grid()

NOTICE dfc engine.py:save_scd merge on=id, updated=0, added=0, deleted=1


,id,name,amount,purchase_date
0,0,Lori,2,2019-04-27
1,1,Stephanie,6,2019-02-06
2,3,Steve,8,2019-04-28


In [16]:
import datetime as dt
version_after_delete = dt.datetime.now()

In [17]:
# modify two
df = df.rows.update([{'id':0, 'name':'Mel'}, {'id':1, 'amount':88}], on='id')
df.data.grid()

,id,name,amount,purchase_date
0,0,Mel,2,2019-04-27
1,1,Stephanie,88,2019-02-06
2,3,Steve,8,2019-04-28


In [18]:
df.save('data/saved/dim', format='scd:parquet', mode='append', merge_on='id')
df_v3 = dfc.load('data/saved/dim', format='scd:parquet', merge_on='id')
df_v3.data.grid()

NOTICE dfc engine.py:save_scd merge on=id, updated=2, added=0, deleted=0


,id,name,amount,purchase_date
0,0,Mel,2,2019-04-27
1,1,Stephanie,88,2019-02-06
2,3,Steve,8,2019-04-28


In [19]:
#redo, nothing happens
df.save('data/saved/dim', format='scd:parquet', mode='append', merge_on='id')
df_v4 = dfc.load('data/saved/dim', format='scd:parquet', merge_on='id')

assert equal(df_v4, df_v3)

NOTICE dfc engine.py:save_scd merge on=id, updated=0, added=0, deleted=0


In [20]:
# all history changes (can be retrieved directly with the underlying format)

df = dfc.load('data/saved/dim', format='parquet')
df.data.grid()

,id,name,amount,purchase_date,_state,_updated
0,0,Mel,2,2019-04-27,0,2019-12-03 05:33:27
1,1,Stephanie,88,2019-02-06,0,2019-12-03 05:33:27
2,0,Lori,2,2019-04-27,1,2019-12-03 05:33:27
3,1,Stephanie,6,2019-02-06,1,2019-12-03 05:33:27
4,0,Lori,2,2019-04-27,0,2019-12-03 05:33:04
5,1,Stephanie,6,2019-02-06,0,2019-12-03 05:33:04
6,2,Linda,6,2019-01-07,0,2019-12-03 05:33:04
7,3,Steve,8,2019-04-28,0,2019-12-03 05:33:11
8,2,Linda,6,2019-01-07,1,2019-12-03 05:33:19


In [21]:
# first version
df = dfc.load('data/saved/dim', format='scd:parquet', merge_on='id', version=0)
assert equal(df, df_v0)

df.data.grid()

,id,name,amount,purchase_date
0,0,Lori,2,2019-04-27
1,1,Stephanie,6,2019-02-06
2,2,Linda,6,2019-01-07


In [22]:
# last version (default)
df = dfc.load('data/saved/dim', format='scd:parquet', merge_on='id', version=-1)
assert equal(df, df_v3)

df.data.grid()

,id,name,amount,purchase_date
0,0,Mel,2,2019-04-27
1,1,Stephanie,88,2019-02-06
2,3,Steve,8,2019-04-28


In [23]:
# second last version
df = dfc.load('data/saved/dim', format='scd:parquet', merge_on='id', version=-2)
assert equal(df, df_v2)

df.data.grid()

,id,name,amount,purchase_date
0,0,Lori,2,2019-04-27
1,1,Stephanie,6,2019-02-06
2,3,Steve,8,2019-04-28


In [24]:
# by date, datetime ( version after delete is version=2)
df = dfc.load('data/saved/dim', format='scd:parquet', merge_on='id', version=version_after_delete)
assert equal(df, df_v2)

df.data.grid()

,id,name,amount,purchase_date
0,0,Lori,2,2019-04-27
1,1,Stephanie,6,2019-02-06
2,3,Steve,8,2019-04-28


In [25]:
# analyze scd history, deleted records, added records, modified records by field
df = dfc.load('data/saved/dim', format='parquet')
df.data.scd_analyze(merge_on='id')

,updated,upd,add,del,changes
0,2019-12-03 05:33:04,0,3,0,"{'name': 0, 'purchase_date': 0, 'amount': 0}"
1,2019-12-03 05:33:11,0,1,0,"{'name': 0, 'purchase_date': 0, 'amount': 0}"
2,2019-12-03 05:33:19,0,0,1,"{'name': 0, 'purchase_date': 0, 'amount': 0}"
3,2019-12-03 05:33:27,2,0,0,"{'name': 1, 'purchase_date': 0, 'amount': 1}"
